In [22]:
import pandas as pd 
import numpy as np
from numpy import random, array
import time 
import os # Dont think this is used 
from multiprocessing import Pool

pd.set_option('display.max_colwidth', -1) # Values in columns won't be shortned 
pd.set_option('chained_assignment',None) # Disabling chained assignments 

geneColID = ["chromosome","source","type","start","end","score","strand","phase","gene_symbol","gene_ensID","length","entrezid"]
geneAnnotationDF = pd.read_csv('entrez_id/geneAnnotationsDF_Selected_entrezID.csv', sep=',', comment='#', low_memory=False, header=0, names=geneColID)
chromosomeColID = ['chromosome','source','type','start','end','score','strand','phase']
chromosomesDF = pd.read_csv('chromosomesDF.csv', sep=',', comment='#', low_memory=False, header=0, names=chromosomeColID)

In [23]:
#Dropping unneccesary stuff and resetting index from 0
geneAnnotationDF = geneAnnotationDF[geneAnnotationDF.gene_symbol != 'CTD-2207O23.3']
chromosomesDF = chromosomesDF.reset_index()
geneAnnotationDF = geneAnnotationDF.reset_index()
chromosomesDF = chromosomesDF.drop(columns=['score', 'strand', 'phase', 'index'])
geneAnnotationDF = geneAnnotationDF.drop(columns=['score', 'phase', 'index'])

In [24]:
geneAnnotationDF = geneAnnotationDF.sort_values(by=['chromosome', 'start'])

In [25]:
# Making a list of DataFrames to be used in future functions in var geneDFL
u = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','MT','X','Y']
geneDFL = []
for c in u:
    geneDF = geneAnnotationDF.loc[geneAnnotationDF['chromosome'] == c]
    geneDF = geneDF.reset_index()
    geneDF = geneDF.drop(columns=['index'])
    geneDF = geneDF.sort_values(by=['start'])
    geneDFL.append(geneDF)

In [26]:
# Making a list of DataFrames to be used in addwindow tss so that if - gene the start is its end and its end is its start 
u = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','MT','X','Y']
geneDFLtss = []
for c in u:
    geneDF = geneAnnotationDF.loc[geneAnnotationDF['chromosome'] == c]
    geneDF = geneDF.reset_index()
    geneDF = geneDF.drop(columns=['index'])
    for i, row in geneDF.iterrows():
        if row['strand'] == '-':
            start = row['end']
            end = row['start']
            geneDF.at[i, 'start'] = start
            geneDF.at[i, 'end'] = end
    geneDF = geneDF.sort_values(by=['start'])
    geneDFLtss.append(geneDF)

In [27]:
# probability that a randomly chosen gene is in a certain chromosome. Ordered from 1 - MT,X,Y | As shown chromosome 1 has highest prob of selection
probabilityL = [0.08069467567425805,
                0.04336798981767917,
                0.043785860443564265,
                0.04319875642790819,
                0.03706910744670193,
                0.0346962654651935,
                0.0330585805012371,
                0.029281524019991866,
                0.02698637835887469,
                0.02605153188596882,
                0.018999829262963137,
                0.07850260746862361,
                0.02088839931657888,
                0.015140187582448091,
                0.016471877931803908,
                0.06427388252370811,
                0.061654572719752214,
                0.058842309890180126,
                0.055363637410522915,
                0.05164908584376297,
                0.04704403712800866,
                0.044858118997812904,
                5.370538628831923e-06,
                0.050577805194942725,
                0.01753760814888535]

In [28]:
dl = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25'] # 23, 24, 25
chrD = {'1': [1, 248956422],'2': [1, 133797422],'3': [1, 135086622],'4': [1, 133275309],'5': [1, 114364328],'6': [1, 107043718],'7': [1, 101991189],'8': [1, 90338345],'9': [1, 83257441],'10': [1, 80373285],'11': [1, 58617616],'12': [1, 242193529],'13': [1, 64444167],'14': [1, 46709983],'15': [1, 50818468],'16': [1, 198295559],'17': [1, 190214555],'18': [1, 181538259],'19': [1, 170805979],'20': [1, 159345973],'21': [1, 145138636],'22': [1, 138394717],'23': [1, 16569],'24': [1, 156040895],'25': [2781480, 56887902]}

def nRandSites(data):
    nSite = data[0]
    rand = data[1] 
    sitesbyC =[[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]

    random.seed(rand)
        
    for site in range(nSite):
        chrN = random.choice(dl, p=probabilityL)
        randnum = round(random.uniform(chrD[chrN][0], chrD[chrN][1]))
        sitesbyC[int(chrN) - 1].append(randnum)
    for l in sitesbyC:
        l.sort()
    return array([array(l) for l in sitesbyC])

def nRandSitesSim(nSite, nSim):
    totalsites = [nSite] * nSim
    chunks = []
    for i in totalsites:
        chunks.append([i])
    for l in chunks:
        l.append(random.randint(100000))
    
    pool = Pool(5)
    result = pool.map(nRandSites, chunks)
    pool.close()
    return(result) 

In [29]:
##### Adds midpoint in gene and upper and lower bound values to geneDFLtss based on size of window or nearby genes. Adding around geneTSS.
def addWindowTSS(window):
    chrD = {'1': [1, 248956422],'2': [1, 133797422],'3': [1, 135086622],'4': [1, 133275309],'5': [1, 114364328],'6': [1, 107043718],'7': [1, 101991189],'8': [1, 90338345],'9': [1, 83257441],'10': [1, 80373285],'11': [1, 58617616],'12': [1, 242193529],'13': [1, 64444167],'14': [1, 46709983],'15': [1, 50818468],'16': [1, 198295559],'17': [1, 190214555],'18': [1, 181538259],'19': [1, 170805979],'20': [1, 159345973],'21': [1, 145138636],'22': [1, 138394717],'23': [1, 16569],'24': [1, 156040895],'25': [2781480, 56887902]}
    geneWindowDFL = geneDFLtss
    outList = []
    chrIDNUM = 0
    for dfG in geneWindowDFL: # ordered by chromosome 1- MT,X,Y
        chrIDNUM += 1 
        dfIndexMax = len(dfG) - 1   
        for i, row in dfG.iterrows():
            if row['strand'] == '+':
                start = row['start']
            if row['strand'] == '-':
                start = row['end']
            dfG.at[i, 'midBody'] = (((row['length'] / 2)) + start) # Middle of gene body 
            if i == 0 or i == dfIndexMax: # Case for start and end genes
                if i == 0:
                    dfG.at[i, 'lowB'] = max(start - window, chrD[str(chrIDNUM)][0]) # chrD[chrIDNUM][0] = start val of the chromosome
                    dfG.at[i, 'upperB'] = min(start + window, ((dfG.iat[i+1, 3] - start) / 2) + start)
                else: # index max case 
                    dfG.at[i, 'lowB'] = max(start - window, start - ((start - dfG.iat[i-1, 3]) / 2)) # dfG.iat[i-1, 3] = start of gene below
                    dfG.at[i, 'upperB'] = min(start + window, chrD[str(chrIDNUM)][1]) # chrD[chrIDNUM][1] = end val of chromosome 
            else: # i > 0 or i < len(geneDF) - 1 
                dfG.at[i, 'lowB'] = max(start - window, start - ((start - dfG.iat[i-1, 3]) / 2)) # dfG.iat[i-1, 3] = start of gene below
                dfG.at[i, 'upperB'] = min(start + window, ((dfG.iat[i+1, 3] - start) / 2) + start) # Start of the gene above position [i,3]
        dfG['midBody'] = dfG['midBody'].astype(int)
        dfG['lowB'] = dfG['lowB'].astype(int)
        dfG['upperB'] = dfG['upperB'].astype(int)
        LBandUB = []
        LBandUB.append(dfG.lowB.values)
        LBandUB.append(dfG.upperB.values)
        outList.append(LBandUB)
    
    return(outList)

In [144]:
# Adds midpoint in gene and upper and lower bound values to geneDFL based on size of window or nearby genes. Adding around geneBODY.
def addWindowBODY(window):
    chrD = {'1': [1, 248956422],'2': [1, 133797422],'3': [1, 135086622],'4': [1, 133275309],'5': [1, 114364328],'6': [1, 107043718],'7': [1, 101991189],'8': [1, 90338345],'9': [1, 83257441],'10': [1, 80373285],'11': [1, 58617616],'12': [1, 242193529],'13': [1, 64444167],'14': [1, 46709983],'15': [1, 50818468],'16': [1, 198295559],'17': [1, 190214555],'18': [1, 181538259],'19': [1, 170805979],'20': [1, 159345973],'21': [1, 145138636],'22': [1, 138394717],'23': [1, 16569],'24': [1, 156040895],'25': [2781480, 56887902]}
    chrIDNUM = 0
    geneWindowDFL = geneDFL
    outList = []
    for dfG in geneWindowDFL: # ordered by chromosome 1- MT,X,Y
        chrIDNUM += 1 
        dfIndexMax = len(dfG) - 1         
        for i, row in dfG.iterrows():
            start = row['start']
            end = row['end']
            dfG.at[i, 'midBody'] = (((row['length'] / 2)) + row['start']) # Middle of gene body 
            
            if i == 0 or i == dfIndexMax: # Case for start and end sites
                if i == 0:
                    dfG.at[i, 'lowB'] = max(start - window, chrD[str(chrIDNUM)][0]) # end of the gene below position
                    dfG.at[i, 'upperB'] = min(end + window, ((dfG.iat[i+1, 3] - end) / 2) + end)
                else: # index max case 
                    dfG.at[i, 'lowB'] = max(start - window, start - ((start - dfG.iat[i-1, 4]) / 2)) 
                    dfG.at[i, 'upperB'] = min(row['end'] + window, chrD[str(chrIDNUM)][1])
            else: # i > 0 or i < len(geneDF) - 1 
                dfG.at[i, 'lowB'] = max(start - window, start - ((start - dfG.iat[i-1, 4]) / 2)) # end of the gene below position [i,4]
                dfG.at[i, 'upperB'] = min(end + window, ((dfG.iat[i+1, 3] - end) / 2) + end) # Start of the gene above position [i,3]
        dfG['midBody'] = dfG['midBody'].astype(int)
        dfG['lowB'] = dfG['lowB'].astype(int)
        dfG['upperB'] = dfG['upperB'].astype(int)
        LBandUB = []
        LBandUB.append(dfG.lowB.values)
        LBandUB.append(dfG.upperB.values)
        outList.append(LBandUB)
    
    return(outList)

In [156]:
# Adds midpoint in gene and upper and lower bound values to geneDFL based on size of window or nearby genes. Adding around geneBODY.
def addWindowBODY20(window):
    geneWindowDFL = geneDFLtss
    chrD = {'1': [1, 248956422],'2': [1, 133797422],'3': [1, 135086622],'4': [1, 133275309],'5': [1, 114364328],'6': [1, 107043718],'7': [1, 101991189],'8': [1, 90338345],'9': [1, 83257441],'10': [1, 80373285],'11': [1, 58617616],'12': [1, 242193529],'13': [1, 64444167],'14': [1, 46709983],'15': [1, 50818468],'16': [1, 198295559],'17': [1, 190214555],'18': [1, 181538259],'19': [1, 170805979],'20': [1, 159345973],'21': [1, 145138636],'22': [1, 138394717],'23': [1, 16569],'24': [1, 156040895],'25': [2781480, 56887902]}
    chrIDNUM = 0
    for dfG in geneWindowDFL: # ordered by chromosome 1- MT,X,Y
        chrIDNUM += 1 
        dfIndexMax = len(dfG) - 1         
        for i, row in dfG.iterrows():
            start = row['start']
            end = row['end']
            dfG.at[i, 'midBody'] = (((row['length'] / 2)) + row['start']) # Middle of gene body 
            
            if i == 0 or i == dfIndexMax: # Case for start and end sites
                if i == 0:
                    dfG.at[i, 'lowB'] = max(start - window, chrD[str(chrIDNUM)][0]) # end of the gene below position
                    dfG.at[i, 'upperB'] = min(end + window, ((dfG.iat[i+1, 3] - end) / 2) + end)
                else: # index max case 
                    dfG.at[i, 'lowB'] = max(start - window, start - ((start - dfG.iat[i-1, 4]) / 2)) 
                    dfG.at[i, 'upperB'] = min(row['end'] + window, chrD[str(chrIDNUM)][1])
            else: # i > 0 or i < len(geneDF) - 1 
                dfG.at[i, 'lowB'] = max(start - window, start - ((start - dfG.iat[i-1, 4]) / 2)) # end of the gene below position [i,4]
                dfG.at[i, 'upperB'] = min(end + window, ((dfG.iat[i+1, 3] - end) / 2) + end) # Start of the gene above position [i,3]
        dfG['midBody'] = dfG['midBody'].astype(int)
        dfG['lowB'] = dfG['lowB'].astype(int)
        dfG['upperB'] = dfG['upperB'].astype(int)

    
    return geneWindowDFL

In [157]:
newen = addWindowBODY20(100000)

In [158]:
newen[4]

chromosome          source  type      start        end strand  \
0    5          ensembl_havana  gene  92151      189972     +       
1    5          ensembl_havana  gene  191511     195353     +       
2    5          ensembl_havana  gene  218215     196871     -       
3    5          ensembl_havana  gene  218241     256700     +       
4    5          ensembl_havana  gene  271621     353856     +       
5    5          ensembl_havana  gene  304176     438291     +       
7    5          ensembl_havana  gene  443158     471937     +       
6    5          ensembl_havana  gene  443160     441498     -       
8    5          ensembl_havana  gene  473098     466124     -       
9    5          ensembl_havana  gene  524332     473310     -       
10   5          ensembl_havana  gene  612272     667168     +       
11   5          ensembl_havana  gene  693395     659862     -       
12   5          ensembl_havana  gene  766952     710360     -       
13   5          ensembl_havana  gene  850986     795606     -       
15   5          ensembl_havana  gene  892643     919357     +       
14   5          ensembl_havana  gene  892824     850291     -       
16   5          havana          gene  997308     987180     -       
17   5          ensembl_havana  gene  1008829    1038943    +       
18   5          ensembl_havana  gene  1112035    1050376    -       
19   5          havana          gene  1178605    1173141    -       
20   5          ensembl_havana  gene  1201595    1225117    +       
21   5          ensembl_havana  gene  1225355    1246189    +       
22   5          ensembl_havana  gene  1295069    1253147    -       
23   5          ensembl_havana  gene  1345099    1317744    -       
24   5          ensembl_havana  gene  1445430    1392790    -       
25   5          ensembl_havana  gene  1523977    1456480    -       
26   5          ensembl_havana  gene  1801366    1798386    -       
27   5          ensembl_havana  gene  1801400    1816605    +       
28   5          ensembl_havana  gene  1887236    1877413    -       
29   5          havana          gene  1887332    1900493    +       
..  ..             ...           ...      ...        ...   ..       
908  5          ensembl_havana  gene  179634784  179614178  -       
910  5          ensembl_havana  gene  179678628  179730925  +       
909  5          ensembl_havana  gene  179680974  179678583  -       
911  5          ensembl_havana  gene  179732850  179796511  +       
912  5          ensembl_havana  gene  179793980  179796647  +       
914  5          ensembl_havana  gene  179806398  179838078  +       
913  5          ensembl_havana  gene  179806952  179797597  -       
915  5          ensembl_havana  gene  179862173  179835133  -       
916  5          ensembl_havana  gene  179907859  179862066  -       
917  5          ensembl_havana  gene  180072118  179911651  -       
918  5          ensembl_havana  gene  180209153  180100791  -       
919  5          ensembl_havana  gene  180292099  180233143  -       
920  5          ensembl_havana  gene  180353387  180300690  -       
921  5          ensembl_havana  gene  180494412  180578405  +       
922  5          ensembl_havana  gene  180591540  180590103  -       
923  5          ensembl_havana  gene  180649624  180601506  -       
924  5          ensembl_havana  gene  180740099  180739042  -       
925  5          ensembl_havana  gene  180815652  180784782  -       
926  5          havana          gene  180831605  180829954  -       
927  5          ensembl_havana  gene  180861285  180847611  -       
928  5          ensembl_havana  gene  180899077  180950906  +       
929  5          ensembl_havana  gene  180988845  181006727  +       
930  5          ensembl_havana  gene  181040225  181061523  +       
931  5          ensembl_havana  gene  181125304  181124357  -       
932  5          ensembl_havana  gene  181154943  181155890  +       
933  5          ensembl_havana  gene  181205293  181193924  -       
93

In [122]:
# Creating list of all the entrezids 
entrezIDLbody = []
for geneDF in geneDFL:
    entrezIDLbody.append(geneDF.entrezid.values)

In [123]:
# Creating list of all the entrezids 
entrezIDLtss = []
for geneDF in geneDFLtss:
    entrezIDLtss.append(geneDF.entrezid.values)

In [124]:
# CLOSEST Genes potentionally regulated by sites in window with respect to the genes body. k = 1 
def geneReadSites(bsL, geneWindow, method='TSS'): # geneWindow = windows lists
    #     geneWindow = addWindowBODY(window)
    chromosomeI = 0 # 0 == chromosome group 1          
    entrezidOutL = [] # Output list of entrez ids 
    if method == 'TSS':
        entrezIDL = entrezIDLtss
    if method == 'BODY':
        entrezIDL = entrezIDLbody
    if method != 'TSS'and method != 'BODY':
        return 'method must be TSS or BODY'
    
    for bounds in geneWindow:
        lowB = bounds[0]
        upperB = bounds[1]
        geneIDS = entrezIDL[chromosomeI]
        sitesL = bsL[chromosomeI]
        
        for site in sitesL:
            for i in range(len(lowB)):
                if site < lowB[i]:
                    break
                if lowB[i] <= site and upperB[i] > site:
                    entrezidOutL.append(geneIDS[i])
                    lowB = lowB[i+1:] # Getting rid of the gene windows because it has been mapped. Genes at a lower positions are removed casuse they are small
                    upperB = upperB[i+1:]
                    geneIDS = geneIDS[i+1:]
                    break # Done with the current site so break 
        chromosomeI += 1   
    return entrezidOutL